In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tqdm
import random
import torch
import matplotlib.pyplot as plt

from utils.visualization import visualize_sample
from model.utils import create_vlm
from model.utils import VisualTextualTokenization
from data import get_dataset
from visual_tokenizer import get_visual_tokenizer
import json

In [ ]:
# dataset = get_dataset('imagenet', '/share/datasets/imagenet', split='val')
# dataset = get_dataset('coco', '/share/datasets/coco2017', split='val')
dataset = get_dataset('clevr_caption', '/home/dchenbs/workspace/datasets/CLEVR_v1.0', split='val')
# dataset = get_dataset('image_paragraph_captioning', '/home/dchenbs/workspace/datasets/VisualGenome', split='val')


# dataset = get_dataset('sharegpt4v', '/home/dchenbs/workspace/datasets/sharegpt4v/ShareGPT4V/sharegpt4v_instruct_gpt4-vision_cap100k.json', split='val')

# dataset = ShareGPT4V('/home/dchenbs/workspace/datasets/sharegpt4v/ShareGPT4V/sharegpt4v_mix665k_cap23k_coco-ap9k_lcs3k_sam9k_div2k.json', split=split)
# dataset = ShareGPT4V('/home/dchenbs/workspace/datasets/sharegpt4v/ShareGPT4V/share-captioner_coco_lcs_sam_1246k_1107.json', split=split)
# dataset = ShareGPT4V('/home/dchenbs/workspace/datasets/sharegpt4v/ShareGPT4V/sharegpt4v_instruct_gpt4-vision_cap100k.json', split=split)

In [ ]:
checkpoint = "/home/dchenbs/workspace/subobjects-VLM/runs/0922-2018-clevr_caption-directsam_tiny(36)-rgb_pixel-SmolLM-1_7B-Instruct/checkpoint-2187"
model, textual_tokenizer = create_vlm(checkpoint)

model = model.cuda().half().eval()

In [ ]:
image_resolution = 768
max_tokens = 36

# config = json.load(open('configs/visual_tokenizer/patch_6_per_side_random.json'))
# config = json.load(open('configs/visual_tokenizer/patch_8_per_side_raster.json'))
# config = json.load(open('configs/visual_tokenizer/directsam_0424.json'))
config = json.load(open('configs/visual_tokenizer/directsam_tiny.json'))

visual_tokenizer = get_visual_tokenizer(**config, image_resolution=image_resolution, max_tokens=max_tokens)

vl_tokenizer = VisualTextualTokenization(textual_tokenizer, visual_tokenizer)

In [ ]:
n_samples = 10
loss = 0
for _ in tqdm.tqdm(range(n_samples)):
    sample = dataset[random.randint(0, len(dataset))]
    inputs = vl_tokenizer([sample], eval=True)

    with torch.no_grad():
        outputs = model(**inputs)
        loss += outputs['loss'].item()

print(f"Loss: {loss / n_samples}")

In [ ]:
sample = dataset[random.randint(0, len(dataset))]

label = sample['text'].split('<|assistant|>')[1].strip().replace(textual_tokenizer.eos_token, '')
sample['text'] = sample['text'].split('<|assistant|>')[0] + '<|assistant|>'

inputs = vl_tokenizer([sample], eval=True)


inputs_embeds, labels = model.prepare_inputs_embeds(
    inputs['text'], inputs['image'], inputs['masks']
)

outputs = model.generate(
    inputs_embeds=inputs_embeds,
    do_sample=False,
    max_new_tokens=dataset.max_text_tokens,
    eos_token_id = textual_tokenizer.eos_token_id,
    pad_token_id = textual_tokenizer.pad_token_id,
)
prediction = textual_tokenizer.decode(outputs[0], skip_special_tokens=True)

visualize_sample(sample, inputs)
print(label)
print('-' * 80)
print(prediction)